In [1]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
import requests
from lxml import html
import re
import pandas as pd
import numpy as np
import math

## Spotify Scraper

### Connect to Spotify API

In [2]:
client_id = '693faf0bce4f4fb0aa5fc5d8b3cc0b25'
client_secret = 'c2ab38a172f44774a0cfae1d7734130f'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

### Scraper Functions
get_artistID_by_artistName - check for reliability, better search

In [3]:
#not deemed reliable yet
def get_artistID_by_artistName(artistName):
        artist = sp.search(q=artistName, type="artist")
        #print(artist)
        artistObject = artist['artists']['items']
        if len(artistObject) == 0:
            return "NOT IN SPOTIFY"
        else:
            artistID = artistObject[0]['id']
            #print('AID: ', artistID)
            return artistID;
    
def get_genre(artistID):
    #navigate to artist
    artist = sp.artist(artistID)
    #Obtain genre
    genres = artist['genres']
    if len(genres) == 0:
        return 'NONE'
    else:
        return genres

def get_followers(artistID):
    #navigate to artist
    artist = sp.artist(artistID)
    #Obtain follower count
    followers = int(artist['followers']['total'])
    return followers

def get_monthlyListeners(artistID):
    r = requests.get('https://open.spotify.com/artist/{}/about'.format(artistID), timeout=10)
    match = re.search(r'Monthly Listeners', r.text)
    #get monthly listeners starting point
    string = r.text[match.start():]
    #split at colon
    strings = string.split(':')
    strings1 = strings[1].split(',')
    monthlyListeners = strings1[0].split(",")
    #grab second string
    return int(monthlyListeners[0].lstrip())

def get_cities(artistID):
    r = requests.get('https://open.spotify.com/artist/{}/about'.format(artistID), timeout=10)
    match = re.search(r'Where People Listen', r.text)
    #get where people listen starting point
    string = r.text[match.start():]
    #split at colon
    strings = string.split(':')
    strings1 = strings[1].split('"')
    cities = strings1[0].split(",")
    #grab second string
    return cities

## Songkick Scraper

### Connect to Songkick API

In [4]:
# Define API Key
api_key = "FYSeGSNlg6HXIfAr"

### Scraper Functions

get_venueArtists: add in festival/lineup capabilities

get_venueGenres: clean up mapping

get_venueSize: Decide how to use followers/listeners, fix metric (within the 30-60 percentile?)

In [23]:
def get_venueArtists(metroID):
    # request json
    r = requests.get("https://api.songkick.com/api/3.0/metro_areas/"+ str(metroID) +"/calendar.json?apikey=" + str(api_key))
    # parse for events
    events = r.json()["resultsPage"]["results"]["event"]
    # define dictionary for storage
    venue_artist_dict = {}
    #loop through events
    for i in range(len(events)):
        # The artist
        artist = events[i]["performance"][0]["artist"]["displayName"]
        # Venue Name
        venue = events[i]["venue"]["displayName"]
        # if the venue is not already in dict, add it with the artist as its value
        if venue not in venue_artist_dict.keys():
            venue_artist_dict[venue] = [artist]
        # if it is in the dict, add the act to its 
        else:
            if artist not in venue_artist_dict[venue]:
                venue_artist_dict[venue].append(artist)
    return venue_artist_dict

def get_venueGenres(venue_artist_dict):
    venue_genre_dict = {}
    # loop through venues
    for venue in venue_artist_dict.keys():
        # loop through artists at each venue
        genre_list = []
        for artist in venue_artist_dict[venue]:
            #get artist id
            id_return = get_artistID_by_artistName(artist)
            # if its too small of an artist, ignore it
            if id_return == 'NOT IN SPOTIFY':
                artist_genre = 'NONE'
                # get the genre of artists in spotify
            else:
                artist_genre = get_genre(id_return)
                print
                mapped_genre=-1
                # replace the genre with a code... NEED TO CLEAN UP
                if any("funk" in s for s in artist_genre):
                    mapped_genre = 0
                if any("rock" in s for s in artist_genre):
                    mapped_genre = 1 
                if any("punk" in s for s in artist_genre):
                    mapped_genre = 1
                if any("reggae" in s for s in artist_genre):
                    mapped_genre = 2
                if any("r&b" in s for s in artist_genre):
                    mapped_genre = 3
                if any("soul" in s for s in artist_genre):
                    mapped_genre = 3
                if any("rap" in s for s in artist_genre):
                    mapped_genre = 4
                if any("bounce" in s for s in artist_genre):
                    mapped_genre = 4
                if any("pop" in s for s in artist_genre):
                    mapped_genre = 5
                if any("jazz" in s for s in artist_genre):
                    mapped_genre = 6
                if any("electronic" in s for s in artist_genre):
                    mapped_genre = 7
                if any("indie" in s for s in artist_genre):
                    mapped_genre = 8
                if any("country" in s for s in artist_genre):
                    mapped_genre = 9
                if any("cajun" in s for s in artist_genre):
                    mapped_genre = 9
                # add genre to list
                genre_list.append(mapped_genre)
        # assign genres to venue
        venue_genre_dict[venue] = list(set([x for x in genre_list if x != -1]))
    # return dictionary
    return venue_genre_dict

def get_venueSize(venue_artist_dict):
    venue_size_dict = {}
    # loop through venues
    for venue in venue_artist_dict.keys():
        # loop through artists at each venue
        size_list = []
        for artist in venue_artist_dict[venue]:
            #get artist id
            id_return = get_artistID_by_artistName(artist)
            # if its too small of an artist, its size is basically 0
            if id_return == 'NOT IN SPOTIFY':
                size_list.append(0)
            # get their followers
            else:
                artist_followers = get_followers(id_return)
                size_list.append(artist_followers)
        venue_size_dict[venue] = np.mean(np.array(size_list))
    return venue_size_dict

## Matching Functions
Both: Need to figure out a way to clean it up


In [24]:
#
# FUNCTION TO ADD ELEMENTS TO DICT
#
def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], tuple):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = value

#
# GENRE MATCH FUNCTION
# if artist genre identity is a match for venue genre, match (hard constraints)
#

def genre_match(d, genre_of):
    #print('in genre check function')
    genre_filtered_dictionary = {}

    # iterate through artist's given constraints O(n)
    for i in genre_of:
        for key, value in d.items():
            for j in value:
                if i == j:
                    append_value(genre_filtered_dictionary, key, value)
                else:
                    pass

    return genre_filtered_dictionary

def size_match(d,size_of):
    perfect_match_dict={}
    for key,value in d.items():
        num=value
        difference=abs(num-size_of)
        per=difference/size_of
        percentage1=1-per
        if(percentage1>=.4):
            perfect_match_dict[key]= round(percentage1, 2)
            None
    return perfect_match_dict

## New Orleans Example

In [30]:
#use names from class... THIS PART WILL BE CHANGED!

names = ["Vampire Weekend", "The Doobie Brothers", "Greta Van Fleet", "Tank and the Bangas"]

df = pd.DataFrame({"band_name": names})
df["id"] = df.band_name.apply(get_artistID_by_artistName)
df["genre"] = df.id.apply(get_genre)
df["listeners"] = df.id.apply(get_monthlyListeners)
df["followers"] = df.id.apply(get_followers)
#pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth',None)
df['mapped_genre'] = [[] for _ in range(df.shape[0])]
for i in range(df.genre.size):
    if any("funk" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('0')
        
    if any("rock" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('1')
        
    if any("punk" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('1')
        
    if any("reggae" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('2')
        
    if any("r&b" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('3')
        
    if any("rap" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('4')
        
    if any("pop" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('5')
        
    if any("jazz" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('6')
        
    if any("electronic" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('7')
        
    if any("indie" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('8')
        
    if any("country" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('9')
df

,band_name,id,genre,listeners,followers,mapped_genre
0,Vampire Weekend,5BvJzeQpmsdsFp4HGUYUEx,"[baroque pop, indie pop, indie rock, modern ro...",4938703,1633741,"[1, 5, 8]"
1,The Doobie Brothers,39T6qqI0jDtSWWioX8eGJz,"[album rock, art rock, blues rock, classic roc...",5988706,1367886,"[1, 9]"
2,Greta Van Fleet,4NpFxQe2UvRCAjto3JqlSl,"[modern blues rock, modern hard rock, rock]",2489065,1268196,[1]
3,Tank and the Bangas,5cAtakaadWHJLxmGKrKcX7,[new orleans funk],284879,146771,[0]


In [31]:
#
# OUTPUT SECTION
# change this portion when looking at different bands
#
nola = get_venueArtists("11772")
nola_venue_genres = get_venueGenres(nola)
nola_venue_sizes = get_venueSize(nola)
for i in range(0,len(df)):
    
    # output user artist name
    print("\n\n\nARTIST:", df.band_name[i])
    
    # output user artist genre
    artist_genre = []
    for j in range(0,len(df.mapped_genre[i])):
        artist_genre.append(int(df.mapped_genre[i][j]))
    print("Artist Genre: ", artist_genre)
    
    # output user artist genre
    artist_size = df.followers[i]
    print("Artist Size (based on followers): ", artist_size)

    # output genre matches
    genre_matches = genre_match(nola_venue_genres ,artist_genre)
    print('genre matches: ', genre_matches)
    
    # output size matches
    size_matches = size_match(nola_venue_sizes ,artist_size)
    print('size matches: ', size_matches)
    




ARTIST: Vampire Weekend
Artist Genre:  [1, 5, 8]
Artist Size (based on followers):  1633741
genre matches:  {"Tipitina's": [[0, 1, 8], [0, 1, 8]], 'UNO Lakefront Arena': [1], 'Smoothie King Center': [1], 'One Eyed Jacks': [[8, 5], [8, 5]], 'Civic Theatre': [5], 'House of Blues New Orleans presented by Cricket Wireless': [8, 4], 'Fillmore New Orleans': [8]}
size matches:  {'UNO Lakefront Arena': 0.54, 'Smoothie King Center': 0.89, 'Fillmore New Orleans': 0.66}



ARTIST: The Doobie Brothers
Artist Genre:  [1, 9]
Artist Size (based on followers):  1367886
genre matches:  {"Tipitina's": [0, 1, 8], 'UNO Lakefront Arena': [1], 'Smoothie King Center': [1], 'Unknown venue': [9], 'Music Box Village': [9, 3, 4, 6], 'Joy Theater': [9]}
size matches:  {'UNO Lakefront Arena': 0.65, 'House of Blues New Orleans presented by Cricket Wireless': 0.47, 'Smoothie King Center': 0.94, 'Fillmore New Orleans': 0.79}



ARTIST: Greta Van Fleet
Artist Genre:  [1]
Artist Size (based on followers):  1268196
g